### 경찰청 사이트 크로링하여 전국 경찰청 이름과 주소 받아오기

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
url='https://www.police.go.kr/index.do'
driver=webdriver.Chrome()
driver.get(url)

In [3]:
# 전체 메뉴 열기
driver.find_element(By.CSS_SELECTOR, '.navbar-toggle').click()

In [4]:
# 기관 소개 열기
driver.find_element(By.XPATH,'//*[@id="main-menu"]/ul/li[6]/a').click()

In [5]:
# 전국경찰관서안내 클릭
driver.find_element(By.XPATH,'//*[@id="main-menu"]/ul/li[6]/ul/li[2]/ul/li[3]/a').click()

In [6]:
# 서울 경찰청 정보 획득
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.table.table-bordered.table-striped.table-hover > tbody > tr')
len(lis)

32

In [7]:
li=lis[0]
li

<tr>
<td class="show-col">서울특별시경찰청</td>
<td class="show-col">서울시 종로구 사직로8길 31 </td>
<td class="show-col">
<a href="https://www.smpa.go.kr/" target="_blank" title="새창열림">https://www.smpa.go.kr/</a>
</td>
</tr>

In [8]:
# 경찰청 이름, 주소 불러오기
name=li.select('.show-col')[0].text.strip()
addr=li.select('.show-col')[1].text.strip()
name, addr

('서울특별시경찰청', '서울시 종로구 사직로8길 31')

In [9]:
# 전국 경찰청 정보 획득
name=[]
addr=[]
lines=[]
for i in range(1,len(soup.select('.nav.nav-tabs.ui-multi-line > li'))+1):
    driver.find_element(By.XPATH,f'//*[@id="tab_{i}"]').click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.table.table-bordered.table-striped.table-hover > tbody > tr')
    for li in lis:
        name=li.select('.show-col')[0].text.strip()
        addr=li.select('.show-col')[1].text.strip()
        lines.append([name, addr])

In [10]:
lines

[['서울특별시경찰청', '서울시 종로구 사직로8길 31'],
 ['서울강남경찰서', '서울시 강남구 테헤란로 114길 11'],
 ['서울강동경찰서', '서울시 강동구 성내로 57'],
 ['서울강북경찰서', '서울시 강북구 오패산로 406'],
 ['서울강서경찰서', '서울시 강서구 화곡로 308'],
 ['서울관악경찰서', '서울시 관악구 관악로5길 33'],
 ['서울광진경찰서', '서울시 광진구 자양로 167'],
 ['서울구로경찰서', '서울시 구로구 가마산로 235'],
 ['서울금천경찰서', '서울시 금천구 시흥대로73길 50'],
 ['서울남대문경찰서', '서울시 중구 한강대로 410'],
 ['서울노원경찰서', '서울시 노원구 노원로 283'],
 ['서울도봉경찰서', '서울시 도봉구 노해로 403'],
 ['서울동대문경찰서', '서울시 동대문구 약령시로21길 29'],
 ['서울동작경찰서', '서울시 동작구 노량진로 148'],
 ['서울마포경찰서', '서울시 마포구 마포대로 183'],
 ['서울방배경찰서', '서울시 서초구 동작대로 204'],
 ['서울서대문경찰서', '서울 서대문구 통일로 113'],
 ['서울서부경찰서', '서울시 은평구 진흥로 58'],
 ['서울서초경찰서', '서울시 서초구 반포대로 179'],
 ['서울성동경찰서', '서울시 성동구 왕십리광장로 9'],
 ['서울성북경찰서', '서울 성북구 보문로 170'],
 ['서울송파경찰서', '서울시 송파구 중대로 221'],
 ['서울수서경찰서', '서울시 강남구 개포로 617'],
 ['서울양천경찰서', '서울시 양천구 목동동로 99'],
 ['서울영등포경찰서', '서울시 영등포구 국회대로 608'],
 ['서울용산경찰서', '서울시 용산구 원효로89길 24'],
 ['서울은평경찰서', '서울시 은평구 연서로 365'],
 ['서울종로경찰서', '서울시 종로구 율곡로 46'],
 ['서울종암경찰서', '서울시 성북구 종암로 135'],
 ['서울중랑경찰서', '서울시

In [11]:
# Dataframe으로 만들기
df=pd.DataFrame(lines, columns=['경찰청이름','주소'])
df

,경찰청이름,주소
0,서울특별시경찰청,서울시 종로구 사직로8길 31
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11
2,서울강동경찰서,서울시 강동구 성내로 57
3,서울강북경찰서,서울시 강북구 오패산로 406
4,서울강서경찰서,서울시 강서구 화곡로 308
...,...,...
272,합천경찰서,경남 합천군 합천읍 황강체육공원로 67
273,제주특별자치도경찰청,제주시 수목원 서길 37
274,서귀포경찰서,서귀포시 신중로 27
275,제주동부경찰서,제주시 동광로 66


In [12]:
df.to_csv('전국경찰청.csv', index=False)

### 전국 경찰청 지도에 표시하기

In [13]:
import folium
from folium.plugins import MarkerCluster
from urllib.parse import quote
import json

In [15]:
with open ("C:\workspace\crawling\kakaokey.txt") as file:
    kakao_key=file.read()

In [16]:
local_url='https://dapi.kakao.com/v2/local/search/address.json'
header={'Authorization': f'KakaoAK {kakao_key}'}

In [17]:
# 경찰청 위도, 경도
lng_list, lat_list=[], []
for i in df.index:
    url_addr=f'{local_url}?query={quote(df.주소[i])}'
    result=requests.get(url_addr, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.경찰청이름[i])

서울종암경찰서
밀양경찰서


In [18]:
df[df.경찰청이름.isin(['서울종암경찰서'])].주소

28    서울시 성북구 종암로 135
Name: 주소, dtype: object

In [19]:
# 주소 변경
df.주소[28]='서울시 성북구 화랑로7길 32'

In [20]:
df[df.경찰청이름.isin(['밀양경찰서'])].주소

258    경도 밀양시 상남면 밀양대로 1545
Name: 주소, dtype: object

In [21]:
# 오타 수정
df.주소[258]='경남 밀양시 상남면 밀양대로 1545'

In [22]:
# 주소 변경 후 재시도
lng_list, lat_list=[], []
for i in df.index:
    url_addr=f'{local_url}?query={quote(df.주소[i])}'
    result=requests.get(url_addr, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.경찰청이름[i])

In [23]:
df['위도']=lat_list
df['경도']=lng_list
df.head()

,경찰청이름,주소,위도,경도
0,서울특별시경찰청,서울시 종로구 사직로8길 31,37.574946,126.971980
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,37.509367,127.067100
2,서울강동경찰서,서울시 강동구 성내로 57,37.528678,127.126957
3,서울강북경찰서,서울시 강북구 오패산로 406,37.637170,127.027042
4,서울강서경찰서,서울시 강서구 화곡로 308,37.551558,126.849656


In [26]:
filename='C:\workspace\crawling\skorea_municipalities_geo_simple.json'
geo_str=json.load(open(filename, encoding='utf8'))

In [27]:
mc = MarkerCluster()
map=folium.Map(location=[37.5502, 126.982], zoom_start=10)
for i in df.index:
    mc.add_child(folium.Marker(
        location=[df.위도[i], df.경도[i]],
        tooltip=f'{df.경찰청이름[i]}',
        popup=folium.Popup(df.주소[i], max_width=200),
        icon=folium.Icon(color='blue', icon='star')
    ))
title='<h3 align="center" style="font-size:20px">전국 경찰청 위치</h3>'
map.get_root().html.add_child(folium.Element(title))
map.add_child(mc)